# El método de Newton para intervalos

## Extensión literal del método de Newton

Supongamos, igual que en el notebook anterior, que $f: \mathbb{R} \to \mathbb{R}$ es una función $C^1$ (continuamente diferenciable) y que tiene una raíz $x^*$, es decir que $f(x^*)=0$. 

Sabemos que los intervalos pueden proveer una forma garantizada de acotar conjuntos. ¿Será posible que se puedan utilizar los intervalos de alguna forma para mejorar el método de Newton para ubicar las raíces de $f$?

**[1]** La primera opción que se le podría ocurrir a uno es ocupar intervalos en el método de Newton tradicional.

(i) Impleméntalo. [Debería ser ¡trivial utilizando tu código del notebook anterior!] 

(ii) Calcula el diámetro de los intervalos $x_i$ resultantes durante la iteración. ¿Qué observas? ¿Por qué ocurre?

In [1]:
include("Intervalos.jl")
include("Duales.jl")
using .Intervalos
using .Duales
using Base.Test

In [2]:
f(x) = x^2 - 2
X0 = Intervalo(0.5, 1.5)

@testset "Pruebas Intervalos, derivada" begin
    @test Intervalo(1) == Intervalo(1.0, 1.0)
    @test Intervalo(1,2) + Intervalo(10,40) == Intervalo(11.0, 42.0)
    @test typeof(Intervalo(1,2)) <: Real
    @test f(X0) == Intervalo(-1.75, 0.25)
    @test derivada(f, X0) == Intervalo(1.0, 3.0)
end

Test Summary:                | Pass  Total
Pruebas Intervalos, derivada |    5      5


Base.Test.DefaultTestSet("Pruebas Intervalos, derivada", Any[], 5, false)

In [3]:
function newton(f, X0::Intervalo, n::Int64 = 10, tolerancia::Float64 = 1e-10)
    for i in 1:n
        X0 = X0 - X0/derivada(f ,X0)
        diametro = abs(X0.sup - X0.inf)
        diametro == 0.0 ? break : nothing
        println(" Diametro del intervalo: ", diametro)
    end
    X0
end

newton (generic function with 3 methods)

In [4]:
newton(f, X0) 

 Diametro del intervalo: 2.333333333333333
 Diametro del intervalo: Inf
 Diametro del intervalo: Inf
 Diametro del intervalo: Inf
 Diametro del intervalo: Inf
 Diametro del intervalo: Inf
 Diametro del intervalo: Inf
 Diametro del intervalo: Inf
 Diametro del intervalo: Inf
 Diametro del intervalo: Inf


Intervalos.Intervalo(-Inf, Inf)

Vemos que implementar el método de Newton tal cual no converge a la raíz

## El método de Newton para intervalos

Está claro que el método de Newton usual no nos sirve con intervalos. De hecho, es común que los métodos numéricos tradicionales no funcionan con intervalos. Sin embargo, al reescribir el método de otra manera, es posible que funcione mejor para acotar las cantidades deseadas.

En los 60s se encontró una manera de *mejorar* el método de Newton, para proveer un método que *garantizadamente* pueda resolver el siguiente problema:

> Dados un intervalo $X \subseteq \mathbb{R}$ y una función $f$ suficientemente suave ($C^1$), encuentra *todas* las raíces $f$ dentro del intervalo $X$.

Es decir, el método de Newton para intervalos nos reportará cuáles son los intervalos dentro de los cuales puede garantizarse que existe una raíz, y que esta raíz es única dentro del intervalo. Asimismo, podría haber intervalos en los cuales no se puede garantizar que haya o no haya una raíz. Cualquier región de $X$ que no esté contenida en la lista de intervalos garantizadamente *no* contiene una raíz de $f$.

Necesitaremos extensiones intervalares $F$ de la función $f$ y $F'$ de su derivada $f'$; la segunda se puede obtener utilizando la diferenciación automática.

**[2]**
Sea $x^*$ una raíz de $f$ y $x$ algún punto del intervalo $X:=[a,b]$. 

(i) Escribe el teorema del valor medio que relaciona $f(x)$ y $f(x^*)$.

* Sea $f:X \longrightarrow \mathbb{R}$ tal que cumple las hipótesis del teorema del valor medio, entonces tenemos: $$f'(\xi) = \dfrac{f(b) - f(a)}{b - a}$$

* Ahora $x^*$ tal que $f(x^*) = 0$ y $x\in X$ con $[x^*,x]\subseteq X$, entonces existe $\xi \in X$ entonces $$f'(\xi) = \dfrac{f(x^*) - f(x)}{x^* - x}$$ 

(ii) De ahí, obtén una expresión *exacta* para $x^*$ en términos de $x$ y un valor desconocido.

* Despejando $x^*$ obtenemos $$x^* = x -\dfrac{f(x)}{f'(\xi)}$$ con $f'(\xi) \neq 0$

(iii) ¿Qué se puede hacer con el valor desconocido?

* Le asignamos todo el intervalo $\xi = X $

(iv) Define un operador $N_f(X, x)$ como el resultado de (iii).

* $N_f(X, x) := x -\dfrac{f(x)}{F'(X)}$

(v) Demuestra que si $x^* \in X$, entonces $x^* \in N_f(X, x)$, y entonces $x^* \in X \cap N_f(X, x)$.

* Puesto que $x^* \in X$ podemos evaluar en el operador $N_f(X, x^*) = x^* -\dfrac{f(x^*)}{F'(X)} = x^* \Longrightarrow x^* \in  N_f(X, x)\cap X$ con $0\notin F'(X)$


Sea $m(X) := \mathrm{mid}(X)$ el punto medio de $X$. Definimos el **operador de Newton** $N_f(X) := N_f(X, m)$. [Es necesario utilizar el intervalo $[m, m]$ cuando se calcula $f(m)$.]

Ahora podemos definir una sucesión de intervalos a partir de un intervalo inicial $X_0$, dada por $X_{k+1} := X_k \cap N_f(X_k)$. Por construcción, si $x^*\in X_0$ entonces $x^*\in X_k$ para toda $k$. Entonces, si $X_0$ contiene una raíz, la raíz se mantiene dentro de la secuencia de intervalos.

Entonces, si pudiéramos controlar de alguna manera que el diámetro de los intervalos $x_k$ disminuya, obtendríamos cotas precisas para $x^*$. Resulta que sí es el caso; la demostración se encuentra, por ejemplo, en el libro de Tucker. Aquí veremos numéricamente que sí es el caso.

**[3]** (i) Escribe una función que acepte una función $f$ y calcule el operador de Newton $N_f(X)$ para un intervalo $X$.

(ii) Escoge una función que tenga una raíz y tal que $0 \notin F'(X)$. Implementa la iteración del método de Newton intervalar y dibuja la secuencia de intervalos que se obtiene.

(iii) Calcula sus diámetros. ¿Qué ocurre?
* Para `Float64`  el diametro converge a $0.0$

In [5]:
function mid(X::Intervalo, centrado::Bool = true) 
    centrado ? dx = 0.5 : dx = 0.5 + (1/2^10)
    abs(X.sup - X.inf)*dx  + X.inf
end

mid (generic function with 2 methods)

In [6]:
diametro(X::Intervalo) = abs(X.sup - X.inf)

diametro (generic function with 1 method)

In [7]:
function OperadorNewton(f, X0::Intervalo)
    x0 = mid(X0)
    f(x0) == 0.0 ? x0 = mid(X0, false): nothing
    
    Nf = x0 - f(x0)/derivada(f ,X0)
end

OperadorNewton (generic function with 1 method)

In [8]:
f(x) = x^3
X0 = Intervalo(-1, 1)
OperadorNewton(f, X0)

Intervalos.Intervalo(-Inf, Inf)

In [9]:
import Base: ∩, ∈, ∉

function ∈(c::Real, x::Intervalo)
    if x.inf <= c <= x.sup
        true
    else
        #@assert x.inf <= c <= x.sup " No es elemento del Intervalo"
        false
    end
    
end

function ∉(c::Real, x::Intervalo)
    if  c < x.inf || c > x.sup
        true
    else
        false
    end
end

function ∩(x::Intervalo, y::Intervalo)
    try
        X = Intervalo(max(x.inf, y.inf), min(x.sup, y.sup))
        
        if X.inf ∈ x && X.inf ∈ y && X.sup ∈ x && X.sup ∈ y
            X
        else
            X = Intervalo(NaN, NaN)
        end
        
    catch 
        X = Intervalo(NaN, NaN)
    end
end


intersect (generic function with 18 methods)

In [10]:
I = Intervalo(1,5) ∩ Intervalo(6,9)

@testset "Pruebas de ∈ y ∩" begin
    
    @test Intervalo(1,5) ∩ Intervalo(3,6) == Intervalo(3.0, 5.0)
    @test Intervalo(1,5) ∩ Intervalo(6,9) == Intervalo(NaN, NaN)
    @test I == Intervalo(NaN, NaN)
    @test Intervalo(1.0, 5.0) ∩ Intervalo(NaN, NaN) == Intervalo(NaN, NaN)
    @test Intervalo(1.0, 5.0) ∩ I == Intervalo(NaN, NaN)
    @test 4.0 ∈ Intervalo(1.0, 5.0)
    @test 5 ∈ Intervalo(1.0, 5.0)
    @test 4.0 ∉ Intervalo(1.0, 3.0) 
    
end

Test Summary:    | Pass  Total
Pruebas de ∈ y ∩ |    8      8


Base.Test.DefaultTestSet("Pruebas de ∈ y ∩", Any[], 8, false)

In [11]:
function newtonIntervalo(f, X0::Intervalo, n::Int64 = 10, tolerancia::Float64 = 1e-10 )
    Nf = Intervalo(0,0)
    for i in 1:n
        Nf = OperadorNewton(f, X0)
        X0 = Nf ∩ X0
        (X0 == Intervalo(NaN,NaN)) && break
        (diametro(X0) == 0.0) && break
    end
#     @show "Diametro del interva lo " abs(X0.sup - X0.inf)
    X0
end

newtonIntervalo (generic function with 3 methods)

In [12]:
f(x) = x^2 - 2
X0 = Intervalo(-3, -2)
X1 = Intervalo(-3, -1)
@show newtonIntervalo(f, X0, 11)
@show newtonIntervalo(f, X1, 11);

newtonIntervalo(f, X0, 11) = Intervalos.Intervalo(NaN, NaN)
newtonIntervalo(f, X1, 11) = Intervalos.Intervalo(-1.4142135623730951, -1.4142135623730951)


**[4]** Podemos entender la acción del operador gráficamente: consiste en tomar rectas con *todas las pendientes dentro del intervalo $F'(X)$* e intersectarlos con el eje $x$. 

(i) Dibuja esto gráficamente.

(ii) Dibuja lo que ocurre en la evolución de la iteración de Newton.

In [13]:
using Plots
gr()

Plots.GRBackend()

In [14]:
function mrecta(f, X::Intervalo)
    xs = collect(linspace(X.inf, X.sup, 50))
    ys = zeros(50)
    x0 = mid(X)
    recta(x::Array{Float64,1}, m::Float64, x0, f) = m.*(x .- x0) .+ f(x0)
    xp = derivada(f, X)
    xpsinf = recta(xs,xp.inf,x0,f)
    xpssup = recta(xs,xp.sup,x0,f)
    xs, xpsinf, xpssup
    
end

mrecta (generic function with 1 method)

In [15]:
f(x) = x^3 - 4
X = Intervalo(1,3)
xs = linspace(X.inf, X.sup, 50)

plot(xlim = (-0.1,3), ylim = (-3, 24),xtick = 0:0.2:3, yticks = -2:2:26 )
plot!(xs, f.(xs),  color = :grey ,label = "f(x) = x^3 -4")
vline!([0,0], color = :black, label = "")
hline!([0,0], color = :black, label = "")


for i in 1:5
    a,b,c = rand(), rand(), rand()
    x0 = mid(X)
    scatter!((x0, f(x0)), markersize = 3, color=RGB(a,b,c), label="")

    xs, xpsinf,xpssup = mrecta(f, X)
    plot!(xs, xpsinf, line=:dash, color=RGB(a,b,c), label="")
    plot!(xs, xpssup, line=:dash, color=RGB(a,b,c), label="")
    
    X = newtonIntervalo(f, X,1)

end
plot!()

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 1.2 
 
 
 1.4 
 
 
 1.6 
 
 
 1.8 
 
 
 2.0 
 
 
 2.2 
 
 
 2.4 
 
 
 2.6 
 
 
 2.8 
 
 
 3.0 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 12 
 
 
 14 
 
 
 16 
 
 
 18 
 
 
 20 
 
 
 22 
 
 
 24 
 
<polyline clip-path="url(#clip02)" style="stroke:#808080; stroke-width:1; stroke-opacity:1; fill:none" points="
 235.509,376.378 242.787,374.638 250.065,372.755 257.343,370.725 264.621,368.542 271.899,366.201 279.177,363.695 286.455,361.019 293.732,358.167 301.01,355.135 
 308.288,351.916 315.566,348.505 322.844,344.897 330.122,341.085 337.4,337.064 344.678,332.829 351.956,328.375 359.233,323.694 366.511,318.782 373.789,313.634 
 381.067,308.244 388.345,302.606 395.623,296.714 402.901,290.563 410.179,284.148 417.457,277.463 424.734,270.502 432.012,263.261 439.29,255.732 446.568,247.911 
 453.846,239.792 461.124,231.369 468.402,222.638 475.68,213.592 482.958,204.225 490.235,194.533 497.513,184.51 504.791,174.15 512.069,163.447 519.347,152.396 
 526.625,140.992 533.903,129.228 541.181,117.1 548.459,104.602 555.737,91.7272 563.014,78.4714 570.292,64.8285 577.57,50.7931 584.848,36.3596 592.126,21.5223 
 
 "/>
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#8dc940; stroke-width:1; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 235.509,321.785 242.787,320.114 250.065,318.442 257.343,316.771 264.621,315.1 271.899,313.429 279.177,311.757 286.455,310.086 293.732,308.415 301.01,306.744 
 308.288,305.073 315.566,303.401 322.844,301.73 330.122,300.059 337.4,298.388 344.678,296.716 351.956,295.045 359.233,293.374 366.511,291.703 373.789,290.032 
 381.067,288.36 388.345,286.689 395.623,285.018 402.901,283.347 410.179,281.676 417.457,280.004 424.734,278.333 432.012,276.662 439.29,274.991 446.568,273.319 
 453.846,271.648 461.124,269.977 468.402,268.306 475.68,266.635 482.958,264.963 490.235,263.292 497.513,261.621 504.791,259.95 512.069,258.278 519.347,256.607 
 526.625,254.936 533.903,253.265 541.181,251.594 548.459,249.922 555.737,248.251 563.014,246.58 570.292,244.909 577.57,243.237 584.848,241.566 592.126,239.895 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#8dc940; stroke-width:1; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 235.509,649.344 242.787,634.303 250.065,619.262 257.343,604.221 264.621,589.18 271.899,574.139 279.177,559.098 286.455,544.057 293.732,529.016 301.01,513.975 
 308.288,498.934 315.566,483.893 322.844,468.852 330.122,453.811 337.4,438.77 344.678,423.729 351.956,408.688 359.233,393.647 366.511,378.606 373.789,363.565 
 381.067,348.524 388.345,333.483 395.623,318.442 402.901,303.401 410.179,288.36 417.457,273.319 424.734,258.278 432.012,243.237 439.29,228.196 446.568,213.155 
 453.846,198.115 461.124,183.074 468.402,168.033 475.68,152.992 482.958,137.951 490.235,122.91 497.513,107.869 504.791,92.8277 512.069,77.7867 519.347,62.7457 
 526.625,47.7048 533.903,32.6638 541.181,17.6228 548.459,2.58182 555.737,-12.4592 563.014,-27.5001 570.292,-42.5411 577.57,-57.5821 584.848,-72.6231 592.126,-87.664 
 
 "/>
 
 
<polyline clip-path="url(#clip02)" style="stroke:#76af90; stroke-width:1; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 235.509,367.895 238.609,367.184 241.709,366.472 244.809,365.76 247.909,365.048 251.008,364.336 254.108,363.625 257.208,362.913 260.308,362.201 263.408,361.489 
 266.508,360.777 269.608,360.065 272.707,359.354 275.807,358.642 278.907,357.93 282.007,357.218 285.107,356.506 288.207,355.795 291.306,355.083 294.406,354.371 
 297.506,353.659 300.606,352.947 303.706,352.235 306.806,351.524 309.906,350.812 313.005,350.1 316.105,349.388 319.205,348.676 322.305,347.965 325.405,347.253 
 328.505,346.541 331.604,345.829 334.704,345.117 337.804,344.406 340.904,343.6

Se sabe que el operador de Newton satisface el siguiente teorema fuerte:

> Supongamos que $f(x)$ sea doblemente continua y diferenciable y $N_f(X)$ esté bien definida en el intervalo $X$. Entonces:

> 1. Si $N_f(X)\cap X = \emptyset$, entonces $X$ no contiene ninguna raíz de $f$.

> 2. Si $N_f(X)  \subseteq X$, entonces $X$ contiene exactamente una raíz de $f$.

## División extendida 

Hasta ahora, sólo hemos podido tratar el caso en el cual la derivada $F'(X)$ no contiene $0$, ya que el operador de Newton contiene una división por este intervalo. 

Sin embargo, resulta que también es posible tratar este caso, mediante la llamada "división extendida".

**[5]** Supón que $F'(X)$ sea un intervalo, digamos $F'(X) = [-a, b]$, que contiene $0$ (con $a > 0$ y  $b > 0$). 

(i) Sea $A \subseteq \mathbb{R}$. Recuerda la definición de $1/A$ como conjunto. 
* $\frac{1}{A} =
  \begin{cases}
    [-\infty,-\frac{1}{a}]\cup [\frac{1}{b},\infty]      & \quad \text{si } 0\in A \text{ y } a,b \neq 0  \\
    [1/a, \infty]       & \quad \text{si } 0\in A \text{ y } b=0  \\
    [-\infty,-1/b]       & \quad \text{si } 0\in A \text{ y } a=0  \\
  \end{cases}
$


(ii) Así, evalúa $1 / F'(X)$. 

* $\frac{1}{ F'(X)} =
  \begin{cases}
    [-\infty,-\frac{1}{a}]\cup [\frac{1}{b},\infty]      & \quad \text{si } 0\in A \text{ y } a,b \neq 0  \\
    [1/a, \infty]       & \quad \text{si } 0\in A \text{ y } b=0  \\
    [-\infty,-1/b]       & \quad \text{si } 0\in A \text{ y } a=0  \\
  \end{cases}$

(iii) Define una función que implementa esta "división extendida" de intervalos (o, más bien, inversa extendida). [Pista: ¿Qué tipo de objeto debe regresar esta función?]

In [16]:
import Base: ∪

function ∪(x::Intervalo, y::Intervalo)
    Intervalo(min(x.inf, y.inf), max(x.sup, y.sup))
end

function ⊂(x::Intervalo, y::Intervalo)
    if x.inf ∈ y && x.sup ∈ y 
        if x.inf != y.inf && x.sup == y.sup 
            true
        elseif x.inf == y.inf && x.sup != y.sup 
            true 
        elseif  x.inf != y.inf && x.sup != y.sup 
            true
        elseif x.inf == y.inf && x.sup == y.sup
            true
        end
    elseif x == Intervalo(NaN, NaN) && y != Intervalo(NaN, NaN)
        true
    else
        false
    end
end


⊂ (generic function with 1 method)

In [17]:
@testset "Pruebas ⊂" begin
    
 @test Intervalo(1.0, 2.0) ⊂ Intervalo(1.0, 2.0)
 @test Intervalo(1.0, 2.0) ⊂ Intervalo(0.0, 3.0) 
 @test Intervalo(1.0, 3.0) ⊂ Intervalo(-1.0, 3.0)
 @test (Intervalo(1.0, 2.0) ⊂ Intervalo(3.0, 4.0)) == false
 @test Intervalo(1.0, 2.0) ⊂ Intervalo(0.5, 3.0)
 @test (Intervalo(NaN,NaN) ⊂ Intervalo(NaN,NaN)) == false
 @test Intervalo(NaN,NaN) ⊂ Intervalo(1,3)
    
end;

Test Summary: | Pass  Total
Pruebas ⊂     |    7      7


In [18]:
function inversaExtendida(x::Intervalo)
    output = Array{Intervalo}(0)
    if sign(x.inf) != sign(x.sup) && x.inf != 0.0 && x.sup != 0.0
        push!(output,Intervalo(-Inf, 1/(x.inf)), Intervalo(1/(x.sup), Inf))
    elseif  x.inf == 0.0
        push!(output,Intervalo(1/(x.sup), Inf))
    elseif x.sup == 0.0
        push!(output,Intervalo(-Inf, 1/(x.inf)))
    else
        push!(output,Intervalo(min(1/x.inf, 1/x.sup), max(1/x.inf, 1/x.sup)))
    end
end

inversaExtendida (generic function with 1 method)

In [19]:
function fprimaIntervalo(f, x::Intervalo, N::Int64=11) 
    xI = []
    
    if x.inf == -Inf || x.sup == Inf
        x0 = mid(x)
        x.inf == -Inf ? xs = collect(linspace(x0, x.sup, N)) :  
        xs = collect(linspace(x.inf, x0, N))
    else
        xs = collect(linspace(x.inf, x.sup, N))
    end
    
    for i in 1:N - 1 
        push!(xI , derivada(f, Intervalo(xs[i], xs[i+1])))
    end

    X = xI[1] ∪ xI[2] 
    
    for i in 3:N-1
        X = X ∪ xI[i]
    end
    X
end

H(x) = x^3 - 3x
I = Intervalo(-2, 2)
@show FpI = fprimaIntervalo(H, I), derivada(H, I);

FpI = (fprimaIntervalo(H, I), derivada(H, I)) = (Intervalos.Intervalo(-3.0, 9.0), Intervalos.Intervalo(-15.0, 9.0))


In [20]:
inversaExtendida(Intervalo(-3,3))

2-element Array{Intervalos.Intervalo,1}:
 Intervalos.Intervalo(-Inf, -0.333333)
 Intervalos.Intervalo(0.333333, Inf)  

https://github.com/MikeInnes/Traceur.jl

**[6]** Resulta que el método de Newton sigue funcionando si utilizamos esta división extendida cuándo sea apropiado. 

(i) Impleméntalo para encontrar *todas* las raíces de una función en un intervalo dado.

(ii) Escoge una función con dos raíces. Muestra gráficamente lo que está ocurriendo.

(iii) Nota que hay casos en los cuales no ocurre ninguna de las posibilidades (1) ni (2) en el teorema del método de Newton para intervalos. ¿Qué se puede hacer en este caso?

In [21]:
function OperadorNewton(f,X0::Intervalo, Xp0::Intervalo)
    x0 = mid(X0)
    f(x0) == 0.0 ? x0 = mid(X0, false): nothing
    
    Nf = x0 - f(x0)*Xp0
end

OperadorNewton (generic function with 2 methods)

In [22]:
function newtonIntervalo2(f, X0::Intervalo, n::Int64 = 10, tolerancia::Float64 = 1e-10 )
    roots = Intervalo[]
    
    if 0.0 ∈ derivada(f, X0)
        X01 = [X0, X0]
        for i in 1:2
            Xp01 = inversaExtendida(derivada(f,X0))
            
            for j in 1:n
                if 0.0 ∉ derivada(f, X01[i])
                    Nf1 = OperadorNewton(f, X01[i])
                else
                    Nf1 = OperadorNewton(f, X01[i],Xp01[i])
                end
                X01[i] = Nf1 ∩ X01[i]
                (X01[i] == Intervalo(NaN,NaN)) && break
                (diametro(X01[i]) == 0.0) && break
            end
        end
        push!(roots, X01[1], X01[2])    
    else
        rr = newtonIntervalo(f, X0, n, tolerancia)
        push!(roots, rr)
    end
    roots
end

newtonIntervalo2 (generic function with 3 methods)

In [23]:
f(x) = x^3 -3x 
I = Intervalo(-3, 0.5)
newtonIntervalo2(f, I)

2-element Array{Intervalos.Intervalo,1}:
 Intervalos.Intervalo(0.0, 0.0)          
 Intervalos.Intervalo(-1.73205, -1.73205)

In [24]:
function uniqueRoot(x::Intervalo, y::Intervalo)
    if x != Intervalo(NaN, NaN) && (x.inf == y.inf || x.sup == y.sup)
        -1
    elseif x != Intervalo(NaN, NaN) && x ⊂ y
        1
    elseif x ∩ y == Intervalo(NaN, NaN)
        0
    else
        0
    end
end

uniqueRoot (generic function with 1 method)

In [25]:
@testset "Pruebas uniqueRoot" begin
    
    @test uniqueRoot(Intervalo(1.0, 10), Intervalo(15.0,20)) == 0
    @test uniqueRoot(Intervalo(1.0, 10), Intervalo(1.0,20)) == -1
    @test uniqueRoot(Intervalo(1.0, 10),Intervalo(-21.0, 24.0)) == 1 
    @test uniqueRoot(Intervalo(1.0, 10),Intervalo(1.0, 10.0)) == -1
    
    end;

Test Summary:      | Pass  Total
Pruebas uniqueRoot |    4      4


In [26]:
function bisection(x::Intervalo)
    x0 = mid(x, false)
    Intervalo[Intervalo(x.inf, x0),Intervalo(x0,x.sup)]
end

bisection (generic function with 1 method)

In [27]:
bisection(Intervalo(-10, 1))

2-element Array{Intervalos.Intervalo,1}:
 Intervalos.Intervalo(-10.0, -4.48926)
 Intervalos.Intervalo(-4.48926, 1.0)  

In [28]:
function sinRepetidos(X::Intervalo, Xs::Array{Intervalo,1})
    sinRepetir = true
    for i in 1:length(Xs)
        if Xs[i] == X
            sinRepetir = false
            break
        end
    end
    sinRepetir
end

sinRepetidos (generic function with 1 method)

In [29]:
function newtonIntervalExten(f, X0::Intervalo, roots = Intervalo[], n::Int64 = 10, tolerancia::Float64 = 1e-10 )
    dx = derivada(f, X0)
    
    if 0.0 ∉ dx
        root = newtonIntervalo2(f, X0, n, tolerancia)
        for j in 1:length(root)
            if root[j] != Intervalo(NaN,NaN) && sinRepetidos(root[j], roots)
                push!(roots, root[j])
            end
        end        
    else

        Xp = inversaExtendida(dx)
        Nf = OperadorNewton.(f, X0, Xp)
        X1 = Nf .∩ X0
        i = 1
        while i != length(X1) + 1
            if uniqueRoot(X1[i], X0) == 1
                root = newtonIntervalo2(f, X1[i], n, tolerancia)
                for j in 1:length(root)
                    if root[j] != Intervalo(NaN,NaN) && sinRepetidos(root[j], roots)
                         push!(roots, root[j])
                    end
                end
                i += 1
            elseif uniqueRoot(X1[i], X0) == -1
                Xbis = bisection(X1[i])
                for j in 1:2
                    root = newtonIntervalExten(f, Xbis[j], roots)
                    for k in 1:length(root)
                        if root[k] != Intervalo(NaN,NaN) && sinRepetidos(root[k], roots)
                             push!(roots, root[k])
                        end
                    end
                end
                i += 1
            else uniqueRoot(X1[i], X0) == 0
                deleteat!(X1, i)
            end
        end
    end
    roots
end

newtonIntervalExten (generic function with 4 methods)

In [30]:
f2(x) = x^3 - 3x 
x = linspace(-3.0, 3.0, 101)
plot(x, f2.(x))
hline!([0,0], color = :black, label = "")
vline!([0,0], color = :black, label = "")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 -10 
 
 
 0 
 
 
 10 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,376.378 44.8976,361.967 50.4252,348.207 55.9528,335.083 61.4803,322.582 67.0079,310.692 72.5354,299.399 78.063,288.689 83.5906,278.55 89.1181,268.969 
 94.6457,259.931 100.173,251.423 105.701,243.434 111.228,235.948 116.756,228.953 122.283,222.435 127.811,216.382 133.339,210.78 138.866,205.616 144.394,200.876 
 149.921,196.548 155.449,192.618 160.976,189.072 166.504,185.898 172.031,183.081 177.559,180.61 183.087,178.471 188.614,176.65 194.142,175.134 199.669,173.909 
 205.197,172.964 210.724,172.284 216.252,171.855 221.78,171.666 227.307,171.702 232.835,171.95 238.362,172.398 243.89,173.031 249.417,173.836 254.945,174.801 
 260.472,175.912 266,177.155 271.528,178.518 277.055,179.987 282.583,181.548 288.11,183.19 293.638,184.897 299.165,186.658 304.693,188.459 310.22,190.286 
 315.748,192.126 321.276,193.966 326.803,195.793 332.331,197.594 337.858,199.355 343.386,201.062 348.913,202.704 354.441,204.265 359.969,205.734 365.496,207.097 
 371.024,208.34 376.551,209.451 382.079,210.416 387.606,211.221 393.134,211.854 398.661,212.302 404.189,212.55 409.717,212.586 415.244,212.397 420.772,211.968 
 426.299,211.288 431.827,210.343 437.354,209.118 442.882,207.602 448.409,205.781 453.937,203.642 459.465,201.171 464.992,198.354 470.52,195.18 476.047,191.634 
 481.575,187.704 487.102,183.375 492.63,178.636 498.157,173.472 503.685,167.87 509.213,161.817 514.74,155.299 520.268,148.304 525.795,140.818 531.323,132.829 
 536.85,124.321 542.378,115.283 547.906,105.701 553.433,95.5625 558.961,84.853 564.488,73.5598 570.016,61.6697 575.543,49.1693 581.071,36.0454 586.598,22.2847 
 592.126,7.87402 
 "/>
 
 
 
 
 
 
 
 
 y1

In [31]:
f2(x) = x^3 - 3x

I1 = Intervalo(-3, -2)
I2 = Intervalo(-3, -1.5)
I3 = Intervalo(-3, -0.5)
I4 = Intervalo(-3, 0.5)
I5 = Intervalo(-3, 1.5)
I6 = Intervalo(-3, 2)
I7 = Intervalo(-3, 3)

newtonIntervalExten(f2, I7)

3-element Array{Intervalos.Intervalo,1}:
 Intervalos.Intervalo(-1.73205, -1.73205)
 Intervalos.Intervalo(0.0, 0.0)          
 Intervalos.Intervalo(1.73205, 1.73205)  

In [32]:
ff(x) = (x-4.3)*(x - 3.1)*(x - 2.3)*(x - 0.8)
x = linspace(.0, 5.0, 101)
plot(x, ff.(x))
plot!(ylims=(-5, 25))
hline!([0,0], color = :black, label = "")
vline!([0,0], color = :black, label = "")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 25 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,13.6816 44.8976,46.2694 50.4252,76.6123 55.9528,104.804 61.4803,130.937 67.0079,155.102 72.5354,177.386 78.063,197.877 83.5906,216.659 89.1181,233.815 
 94.6457,249.426 100.173,263.572 105.701,276.329 111.228,287.774 116.756,297.98 122.283,307.019 127.811,314.961 133.339,321.874 138.866,327.824 144.394,332.876 
 149.921,337.093 155.449,340.535 160.976,343.262 166.504,345.33 172.031,346.794 177.559,347.709 183.087,348.126 188.614,348.094 194.142,347.662 199.669,346.875 
 205.197,345.777 210.724,344.412 216.252,342.82 221.78,341.038 227.307,339.105 232.835,337.055 238.362,334.921 243.89,332.735 249.417,330.526 254.945,328.322 
 260.472,326.148 266,324.029 271.528,321.987 277.055,320.041 282.583,318.211 288.11,316.512 293.638,314.961 299.165,313.568 304.693,312.347 310.22,311.305 
 315.748,310.45 321.276,309.788 326.803,309.323 332.331,309.055 337.858,308.986 343.386,309.113 348.913,309.433 354.441,309.94 359.969,310.627 365.496,311.484 
 371.024,312.501 376.551,313.665 382.079,314.961 387.606,316.371 393.134,317.879 398.661,319.464 404.189,321.102 409.717,322.772 415.244,324.446 420.772,326.097 
 426.299,327.696 431.827,329.211 437.354,330.61 442.882,331.856 448.409,332.914 453.937,333.745 459.465,334.307 464.992,334.559 470.52,334.457 476.047,333.954 
 481.575,333.003 487.102,331.553 492.63,329.553 498.157,326.951 503.685,323.689 509.213,319.712 514.74,314.961 520.268,309.374 525.795,302.888 531.323,295.441 
 536.85,286.964 542.378,277.391 547.906,266.65 553.433,254.67 558.961,241.378 564.488,226.697 570.016,210.551 575.543,192.86 581.071,173.544 586.598,152.518 
 592.126,129.699 
 "/>
 
 
 
 
 
 
 
 
 y1

In [33]:
ff(x) = (x-4.3)*(x - 3.1)*(x - 2.3)*(x - 0.7)

for xsup in 0:1:5
    I = Intervalo(0, xsup)
    roots = newtonIntervalExten(ff, I)
    @show I
    @show length(roots), roots
end

I = Intervalos.Intervalo(0.0, 0.0)
(length(roots), roots) = (0, Intervalos.Intervalo[])
I = Intervalos.Intervalo(0.0, 1.0)
(length(roots), roots) = (1, Intervalos.Intervalo[Intervalos.Intervalo(0.7, 0.7)])
I = Intervalos.Intervalo(0.0, 2.0)
(length(roots), roots) = (1, Intervalos.Intervalo[Intervalos.Intervalo(0.7, 0.7)])
I = Intervalos.Intervalo(0.0, 3.0)
(length(roots), roots) = (2, Intervalos.Intervalo[Intervalos.Intervalo(0.7, 0.7), Intervalos.Intervalo(2.3, 2.3)])
I = Intervalos.Intervalo(0.0, 4.0)
(length(roots), roots) = (3, Intervalos.Intervalo[Intervalos.Intervalo(0.7, 0.7), Intervalos.Intervalo(2.3, 2.3), Intervalos.Intervalo(3.1, 3.1)])
I = Intervalos.Intervalo(0.0, 5.0)
(length(roots), roots) = (4, Intervalos.Intervalo[Intervalos.Intervalo(3.1, 3.1), Intervalos.Intervalo(4.3, 4.3), Intervalos.Intervalo(0.7, 0.7), Intervalos.Intervalo(2.3, 2.3)])


In [34]:
f(x) = x^3 - 3x + 5

I = Intervalo(-3, 2)

newtonIntervalExten(f, I)

1-element Array{Intervalos.Intervalo,1}:
 Intervalos.Intervalo(-2.27902, -2.27902)

**[7]** Considera la familia de polinomios de Wilkinson, definidos por $W_n(x) := \prod_{i=1}^n (x-i)$. Utiliza el método de Newton para encontrar todas sus raíces.

In [35]:
function wilkinson(x::Real, n::Int64)
    y = 1
    for i in 1:n
        y *=(x - i)
    end
    return y
end

wilkinson (generic function with 1 method)

In [36]:
n = 6
x = linspace(0.0, n + 1 , 1001)

plot(x, wilkinson.(x, n))
plot!(ylims=(-8, 8))
hline!([0,0], color = :black, label = "")
vline!([0,0], color = :black, label = "")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 -5 
 
 
 0 
 
 
 5 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,-16390.6 39.9228,-16108 40.4756,-15829.1 41.0283,-15553.7 41.5811,-15281.9 42.1339,-15013.7 42.6866,-14748.9 43.2394,-14487.6 43.7921,-14229.8 44.3449,-13975.3 
 44.8976,-13724.2 45.4504,-13476.4 46.0031,-13231.9 46.5559,-12990.7 47.1087,-12752.7 47.6614,-12517.9 48.2142,-12286.3 48.7669,-12057.8 49.3197,-11832.4 49.8724,-11610.1 
 50.4252,-11390.9 50.978,-11174.6 51.5307,-10961.4 52.0835,-10751.1 52.6362,-10543.7 53.189,-10339.2 53.7417,-10137.6 54.2945,-9938.76 54.8472,-9742.78 55.4,-9549.57 
 55.9528,-9359.11 56.5055,-9171.38 57.0583,-8986.33 57.611,-8803.95 58.1638,-8624.21 58.7165,-8447.08 59.2693,-8272.52 59.822,-8100.52 60.3748,-7931.05 60.9276,-7764.08 
 61.4803,-7599.59 62.0331,-7437.54 62.5858,-7277.91 63.1386,-7120.68 63.6913,-6965.82 64.2441,-6813.3 64.7969,-6663.1 65.3496,-6515.19 65.9024,-6369.55 66.4551,-6226.16 
 67.0079,-6084.98 67.5606,-5946 68.1134,-5809.19 68.6661,-5674.52 69.2189,-5541.97 69.7717,-5411.53 70.3244,-5283.15 70.8772,-5156.83 71.4299,-5032.54 71.9827,-4910.25 
 72.5354,-4789.94 73.0882,-4671.59 73.6409,-4555.18 74.1937,-4440.69 74.7465,-4328.08 75.2992,-4217.35 75.852,-4108.46 76.4047,-4001.4 76.9575,-3896.15 77.5102,-3792.68 
 78.063,-3690.97 78.6157,-3591.01 79.1685,-3492.76 79.7213,-3396.22 80.274,-3301.36 80.8268,-3208.16 81.3795,-3116.59 81.9323,-3026.65 82.485,-2938.31 83.0378,-2851.55 
 83.5906,-2766.35 84.1433,-2682.69 84.6961,-2600.55 85.2488,-2519.92 85.8016,-2440.77 86.3543,-2363.09 86.9071,-2286.85 87.4598,-2212.05 88.0126,-2138.66 88.5654,-2066.66 
 89.1181,-1996.03 89.6709,-1926.76 90.2236,-1858.84 90.7764,-1792.23 91.3291,-1726.93 91.8819,-1662.92 92.4346,-1600.18 92.9874,-1538.69 93.5402,-1478.44 94.0929,-1419.42 
 94.6457,-1361.59 95.1984,-1304.96 95.7512,-1249.5 96.3039,-1195.19 96.8567,-1142.03 97.4094,-1089.99 97.9622,-1039.06 98.515,-989.225 99.0677,-940.468 99.6205,-892.775 
 100.173,-846.13 100.726,-800.52 101.279,-755.929 101.831,-712.343 102.384,-669.747 102.937,-628.127 103.49,-587.468 104.043,-547.758 104.595,-508.982 105.148,-471.126 
 105.701,-434.176 106.254,-398.12 106.806,-362.943 107.359,-328.633 107.912,-295.176 108.465,-262.56 109.017,-230.771 109.57,-199.797 110.123,-169.625 110.676,-140.243 
 111.228,-111.639 111.781,-83.799 112.334,-56.7125 112.887,-30.367 113.439,-4.75069 113.992,20.1482 114.545,44.3413 115.098,67.8401 115.65,90.6561 116.203,112.8 
 116.756,134.284 117.309,155.119 117.861,175.315 118.414,194.883 118.967,213.835 119.52,232.18 120.072,249.93 120.625,267.093 121.178,283.682 121.731,299.706 
 122.283,315.174 122.836,330.097 123.389,344.486 123.942,358.348 124.494,371.695 125.047,384.535 125.6,396.878 126.153,408.733 126.706,420.11 127.258,431.017 
 127.811,441.464 128.364,451.459 128.917,461.011 129.469,470.129 130.022,478.822 130.575,487.097 131.128,494.964 131.68,502.43 132.233,509.504 132.786,516.194 
 133.339,522.507 133.891,528.453 134.444,534.038 134.997,539.27 135.55,544.157 136.102,548.707 136.655,552.926 137.208,556.823 137.761,560.404 138.313,563.676 
 138.866,566.648 139.419,569.325 139.972,571.714 140.524,573.823 141.077,575.658 141.63,577.225 142.183,578.532 142.735,579.585 143.288,580.389 143.841,580.952 
 144.394,581.28 144.946,581.378 145.499,581.254 146.052,580.911 146.605,580.358 147.157,579.599 147.71,578.64 148.263,577.487 148.816,576.145 149.369,574.62 
 149.921,572.917 150.474,571.042 151.027,569 151.58,566.796 152.132,564.435 152.685,561.922 153.238,559.262 153.791,556.46 154.343,553.521 154.896,550.45 
 155.449,547.251 156.002,543.928 156.554,540.487 157.107,536.932 157.66,533.267 158.213,529.497 158.765,525.625 159.318,521.656 159.871,517.595 160.424,513.444 
 160.976,509.209 161.529,504.892

In [37]:
n = 6

function wilkinson(x)
    y = 1
    for i in 1:n
        y *= (x - i)
    end
    y
end

rs = newtonIntervalExten(wilkinson, Intervalo(0, n + 1))

6-element Array{Intervalos.Intervalo,1}:
 Intervalos.Intervalo(1.0, 1.0)
 Intervalos.Intervalo(3.0, 3.0)
 Intervalos.Intervalo(2.0, 2.0)
 Intervalos.Intervalo(5.0, 5.0)
 Intervalos.Intervalo(4.0, 4.0)
 Intervalos.Intervalo(6.0, 6.0)

In [38]:
n = 150

function wilkinson(x)
    y = 1
    for i in 1:n
        y *= (x - i)
    end
    y
end

rs = newtonIntervalExten(wilkinson, Intervalo(0, n + 1))

150-element Array{Intervalos.Intervalo,1}:
 Intervalos.Intervalo(21.0, 21.0)  
 Intervalos.Intervalo(19.0, 19.0)  
 Intervalos.Intervalo(20.0, 20.0)  
 Intervalos.Intervalo(23.0, 23.0)  
 Intervalos.Intervalo(22.0, 22.0)  
 Intervalos.Intervalo(25.0, 25.0)  
 Intervalos.Intervalo(26.0, 26.0)  
 Intervalos.Intervalo(24.0, 24.0)  
 Intervalos.Intervalo(27.0, 27.0)  
 Intervalos.Intervalo(28.0, 28.0)  
 Intervalos.Intervalo(29.0, 29.0)  
 Intervalos.Intervalo(30.0, 30.0)  
 Intervalos.Intervalo(31.0, 31.0)  
   ⋮                               
 Intervalos.Intervalo(118.0, 118.0)
 Intervalos.Intervalo(116.0, 116.0)
 Intervalos.Intervalo(123.0, 123.0)
 Intervalos.Intervalo(124.0, 124.0)
 Intervalos.Intervalo(125.0, 125.0)
 Intervalos.Intervalo(127.0, 127.0)
 Intervalos.Intervalo(126.0, 126.0)
 Intervalos.Intervalo(129.0, 129.0)
 Intervalos.Intervalo(128.0, 128.0)
 Intervalos.Intervalo(131.0, 131.0)
 Intervalos.Intervalo(132.0, 132.0)
 Intervalos.Intervalo(130.0, 130.0)